In [ ]:
# notebooks/01_Exploratory_Leadership_Analysis.ipynb

# --- Configuração Inicial do Ambiente (Necessário para imports de src) ---
import sys
import os

# Adiciona o diretório raiz do projeto ao sys.path
# Para notebooks, __file__ não é definido, então usamos um método alternativo
# para encontrar o diretório do notebook e, a partir dele, a raiz do projeto.
current_notebook_dir = os.getcwd() # Obtém o diretório de trabalho atual (deve ser 'notebooks/')
# Sobe um nível de 'notebooks/' para 'transdevs_techexperience/'
project_root = os.path.abspath(os.path.join(current_notebook_dir, '..'))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

# --- Importações Necessárias ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import pickle # Para carregar modelos salvos
import logging

# Configurar o logging para exibir no notebook, se desejar (opcional, notebooks já printam)
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Importa as configurações e funções do nosso projeto
from src.config import (
    EDA_FINAL_PATH, ANONYMIZED_PII_PATH, LEADERSHIP_TYPES, GROUP_NAMES,
    TEXT_COLUMNS_FOR_NLP, TOPIC_MODEL_PATH, TFIDF_VECTORIZER_PATH,
    TOPIC_TO_GROUP_APTITUDE_MAP, POSITIVE_WORDS, NEGATIVE_WORDS
)
from src.analysis.nlp_processing import (
    tokenize_and_lemmatize, clean_text, correct_typos_and_standardize,
    extract_ngrams, vectorize_text_tfidf, apply_topic_modeling_lda, analyze_sentiment
)
from src.analysis.leadership_analysis import (
    load_eda_data, load_pii_mapping # Vamos redefinir a lógica mais avançada aqui no notebook para teste
)

# --- Carregar Dados ---
print("Carregando dados para análise no notebook...")
df_eda = load_eda_data()
df_pii = load_pii_mapping()

if df_eda.empty:
    print("Erro: DataFrame da EDA está vazio. Certifique-se de ter executado 'run_eda.py'.")
    sys.exit() # Sai do notebook se não houver dados

print(f"DataFrame EDA carregado com {len(df_eda)} registros.")
print(f"DataFrame PII carregado com {len(df_pii)} registros (para referência segura).")

# Juntar os nomes para facilitar a visualização no notebook (com segurança, pois é ambiente local)
if not df_pii.empty:
    df_eda_with_names = df_eda.merge(df_pii[['participant_id', 'nome_completo']], on='participant_id', how='left')
else:
    df_eda_with_names = df_eda.copy()
    df_eda_with_names['nome_completo'] = df_eda_with_names['participant_id'].apply(lambda x: f"Participante_{x}")

# --- Análise do Gap de Preferências de Liderança ---

print("\n--- Análise do Gap de Preferências de Liderança ---")

# Identificar os grupos que ainda precisam de líderes diretos (baseado na run_eda.py anterior)
# Aqui, vamos simular a lógica da run_eda.py para saber quais grupos ficaram sem líder direto
direct_leaders_df = df_eda[df_eda['interesse_lideranca'] == LEADERSHIP_TYPES['DIRETA']].copy()
group_leadership_status = {group: {'leader_id': None, 'leader_name': None, 'type': None, 'preferred_by_direct': []} for group in GROUP_NAMES}

for idx, row in direct_leaders_df.iterrows():
    participant_id = row['participant_id']
    pref_group = row['grupo_principal']
    alt_group = row['grupo_alternativo']

    if group_leadership_status[pref_group]['leader_id'] is None:
        group_leadership_status[pref_group]['leader_id'] = participant_id
    elif alt_group != 'Não tenho interesse por nenhuma outra opção' and group_leadership_status[alt_group]['leader_id'] is None:
        group_leadership_status[alt_group]['leader_id'] = participant_id

groups_needing_leaders = [group for group, status in group_leadership_status.items() if status['leader_id'] is None]

print(f"Grupos que ainda precisam de liderança direta: {groups_needing_leaders}")

# Pessoas que querem "ajudar na liderança"
support_leaders_candidates = df_eda_with_names[df_eda_with_names['interesse_lideranca'] == LEADERSHIP_TYPES['SUPORTE']].copy()

print(f"\nTotal de candidatos a líder de suporte: {len(support_leaders_candidates)}")
print("Preferências de grupo dos candidatos a líder de suporte:")
print(support_leaders_candidates[['nome_completo', 'grupo_principal', 'grupo_alternativo']])

# Verificar o alinhamento das preferências dos candidatos de suporte com os grupos carentes
alignment_data = []
for idx, row in support_leaders_candidates.iterrows():
    name = row['nome_completo']
    main_pref = row['grupo_principal']
    alt_pref = row['grupo_alternativo']
    
    main_match = main_pref in groups_needing_leaders
    alt_match = alt_pref in groups_needing_leaders if alt_pref != 'Não tenho interesse por nenhuma outra opção' else False
    
    alignment_data.append({
        'nome_completo': name,
        'grupo_principal_pref': main_pref,
        'grupo_alternativo_pref': alt_pref,
        'main_pref_needs_leader': main_match,
        'alt_pref_needs_leader': alt_match,
        'aligned_to_needing_group': main_match or alt_match
    })

df_alignment = pd.DataFrame(alignment_data)

print("\nAlinhamento dos candidatos a líder de suporte com os grupos que precisam de líderes:")
print(df_alignment)

if not df_alignment['aligned_to_needing_group'].any():
    print("\nCONFIRMADO: Nenhum candidato a líder de suporte tem como preferência direta (principal ou alternativa) os grupos que ainda precisam de líderes.")
    print("Isso explica a mensagem anterior. Precisamos de uma lógica mais avançada para sugerir.")
else:
    print("\nAlguns candidatos a líder de suporte se alinham com os grupos que precisam de líderes. Vamos usar a lógica avançada para sugerir.")


# --- Exploração Detalhada dos Tópicos LDA para Mapeamento ---

print("\n--- Exploração Detalhada dos Tópicos LDA para Mapeamento ---")

# Carregar o vetorizador TF-IDF e o modelo LDA salvos
try:
    with open(TFIDF_VECTORIZER_PATH, 'rb') as f:
        tfidf_vectorizer = pickle.load(f)
    with open(TOPIC_MODEL_PATH, 'rb') as f:
        lda_model = pickle.load(f)
    print("Modelos TF-IDF e LDA carregados com sucesso.")
except FileNotFoundError:
    print("Erro: Modelos TF-IDF ou LDA não encontrados. Certifique-se de ter executado 'run_eda.py'.")
    sys.exit()

feature_names = tfidf_vectorizer.get_feature_names_out()

def display_topics(model, feature_names, no_top_words):
    """Exibe as top palavras para cada tópico."""
    for topic_idx, topic in enumerate(model.components_):
        print(f"Tópico {topic_idx+1}:")
        print(" ".join([f"{feature_names[i]} ({topic[i]:.2f})"
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print("\n")

print(f"\nPalavras-chave dos Tópicos (Top 10 com pesos):")
display_topics(lda_model, feature_names, 10)


# Recalcular a distribuição de tópicos por participante (se não estiver em df_eda)
if 'main_topic' not in df_eda.columns:
    # Gerar textos limpos para vetorização (como no run_eda.py)
    text_for_topic_modeling = df_eda[TEXT_COLUMNS_FOR_NLP].fillna('').agg(' '.join, axis=1).apply(correct_typos_and_standardize).apply(clean_text)
    
    # Vetorizar os textos
    tfidf_matrix = tfidf_vectorizer.transform(text_for_topic_modeling)
    
    # Obter as distribuições de tópico
    topic_distribution = lda_model.transform(tfidf_matrix)
    
    df_eda['main_topic'] = np.argmax(topic_distribution, axis=1) + 1
    for i in range(lda_model.n_components):
        df_eda[f'topic_{i+1}_score'] = topic_distribution[:, i]
    print("Colunas de tópicos adicionadas ao df_eda.")

print("\nDistribuição de Grupos Principais por Tópico Principal (Top 3 por Tópico):")
for topic_id in sorted(df_eda['main_topic'].unique()):
    topic_group_prefs = df_eda[df_eda['main_topic'] == topic_id]['grupo_principal'].value_counts(normalize=True).head(3) * 100
    if not topic_group_prefs.empty:
        print(f"--- Tópico {topic_id} ---")
        print(topic_group_prefs.apply(lambda x: f"{x:.2f}%"))
        print("-" * 20)

print("\nDistribuição de Grupos Alternativos por Tópico Principal (Top 3 por Tópico):")
for topic_id in sorted(df_eda['main_topic'].unique()):
    topic_group_alt_prefs = df_eda[df_eda['main_topic'] == topic_id]['grupo_alternativo'].value_counts(normalize=True).head(3) * 100
    if not topic_group_alt_prefs.empty:
        print(f"--- Tópico {topic_id} ---")
        print(topic_group_alt_prefs.apply(lambda x: f"{x:.2f}%"))
        print("-" * 20)


# --- EXERCÍCIO INTERATIVO: Refinar o Mapeamento Tópico-Grupo ---
print("\n--- EXERCÍCIO: Refine o Mapeamento Tópico-Grupo ---")
print("Ajuste os pesos (0.0 a 1.0) para cada grupo em cada tópico com base nas palavras-chave e nas distribuições acima.")
print("Um peso maior indica maior aptidão do tópico para aquele grupo. Salvaremos este mapa no config.py.")

# Esta é a versão do config.py. Copie para cá e ajuste.
TOPIC_TO_GROUP_APTITUDE_MAP_NOTEBOOK = {
    1: {'G1 - Automações Wix': 0.4, 'G2 - API de Orquestração': 0.4, 'G3 - Integração WhatsApp': 0.4, 'G4 - SUPABASE (Banco de Dados)': 0.4}, # Tópico 1 (Busca Conhecimento) - Aptidão geral
    2: {'G1 - Automações Wix': 0.6, 'G2 - API de Orquestração': 0.6, 'G3 - Integração WhatsApp': 0.6, 'G4 - SUPABASE (Banco de Dados)': 0.6}, # Tópico 2 (Desenvolvimento/Recursos) - Aptidão para organização e processos
    3: {'G1 - Automações Wix': 0.7, 'G2 - API de Orquestração': 0.7, 'G3 - Integração WhatsApp': 0.7, 'G4 - SUPABASE (Banco de Dados)': 0.7}, # Tópico 3 (Comunidade/Inclusão) - Aptidão para foco em pessoas
    4: {'G1 - Automações Wix': 0.3, 'G2 - API de Orquestração': 0.8, 'G3 - Integração WhatsApp': 0.3, 'G4 - SUPABASE (Banco de Dados)': 0.9}, # Tópico 4 (Análise/Gestão/Network) - Forte para G2 (API), G4 (Dados)
    5: {'G1 - Automações Wix': 0.9, 'G2 - API de Orquestração': 0.4, 'G3 - Integração WhatsApp': 0.9, 'G4 - SUPABASE (Banco de Dados)': 0.4}, # Tópico 5 (Projetos/Criação/Colaboração) - Forte para G1 (Wix) e G3 (WhatsApp)
}

print("\n--- Mapeamento Tópico-Grupo Atualizado (Ajuste os pesos no código acima) ---")
for topic_id, group_scores in TOPIC_TO_GROUP_APTITUDE_MAP_NOTEBOOK.items():
    print(f"Tópico {topic_id}: {group_scores}")


# --- Nova Lógica de Potencial de Liderança (para simular e testar aqui) ---
print("\n--- Teste da Nova Lógica de Potencial de Liderança ---")

def calculate_aptitude_score(row, group_needed, topic_to_group_map, sentiment_cols, lda_model_components):
    """Calcula o score de aptidão de um participante para um grupo específico."""
    score = 0.0

    # 1. Alinhamento de Preferência Direta
    if row['grupo_principal'] == group_needed:
        score += 0.5
    elif row['grupo_alternativo'] == group_needed:
        score += 0.3

    # 2. Alinhamento de Tópicos (usando o mapa refinado do notebook)
    main_topic_id = row['main_topic'] if not pd.isna(row['main_topic']) else None
    if main_topic_id is not None and main_topic_id in topic_to_group_map:
        topic_aptitude = topic_to_group_map[main_topic_id].get(group_needed, 0)
        score += topic_aptitude * 0.7 # Peso do alinhamento do tópico

    # 3. Score de Sentimento Ponderado (Proatividade, Engajamento)
    sentiment_score = sum(1 for col in sentiment_cols if col in row and row[col] == 'Positivo') - \
                      sum(1 for col in sentiment_cols if col in row and row[col] == 'Negativo')
    score += sentiment_score * 0.1

    return score

# Simular a atribuição para líderes diretos e identificar grupos carentes
temp_group_leadership_status = {group: {'leader_id': None, 'type': None} for group in GROUP_NAMES}
for idx, row in df_eda.iterrows():
    if row['interesse_lideranca'] == LEADERSHIP_TYPES['DIRETA']:
        participant_id = row['participant_id']
        pref_group = row['grupo_principal']
        alt_group = row['grupo_alternativo']
        
        assigned = False
        if temp_group_leadership_status[pref_group]['leader_id'] is None:
            temp_group_leadership_status[pref_group]['leader_id'] = participant_id
            temp_group_leadership_status[pref_group]['type'] = 'Direta (Principal)'
            assigned = True
        elif alt_group != 'Não tenho interesse por nenhuma outra opção' and temp_group_leadership_status[alt_group]['leader_id'] is None:
            temp_group_leadership_status[alt_group]['leader_id'] = participant_id
            temp_group_leadership_status[alt_group]['type'] = 'Direta (Alternativa)'
            assigned = True
        
        if assigned:
            df_eda.loc[df_eda['participant_id'] == participant_id, 'sim_status_lideranca_final'] = f"Líder Direto Atribuído ({temp_group_leadership_status[pref_group]['type']})"
            df_eda.loc[df_eda['participant_id'] == participant_id, 'sim_sugestao_lideranca_grupo'] = pref_group # Ou alt_group, dependendo
        else:
             df_eda.loc[df_eda['participant_id'] == participant_id, 'sim_status_lideranca_final'] = "Líder Direto (sem atribuição por conflito)"
             df_eda.loc[df_eda['participant_id'] == participant_id, 'sim_sugestao_lideranca_grupo'] = 'N/A'


groups_needing_leaders_sim = [group for group, status in temp_group_leadership_status.items() if status['leader_id'] is None]
print(f"\nGrupos simulados ainda sem liderança direta: {groups_needing_leaders_sim}")

# Filtrar e calcular scores para potenciais líderes de suporte
potential_support_sim_insights = []
for idx, row in df_eda.iterrows():
    if row['interesse_lideranca'] == LEADERSHIP_TYPES['SUPORTE']:
        participant_id = row['participant_id']
        best_aptitude_score = -1
        best_suggested_group = 'N/A'

        for group_needed in groups_needing_leaders_sim:
            score = calculate_aptitude_score(row, group_needed, TOPIC_TO_GROUP_APTITUDE_MAP_NOTEBOOK, ['objetivo_proposito_sentiment', 'bagagem_contribuicao_sentiment', 'compromisso_pessoal_sentiment'], lda_model.components_)
            if score > best_aptitude_score:
                best_aptitude_score = score
                best_suggested_group = group_needed
        
        if best_aptitude_score > 0:
            potential_support_sim_insights.append({
                'participant_id': participant_id,
                'nome_completo': df_eda_with_names.loc[df_eda_with_names['participant_id'] == participant_id, 'nome_completo'].iloc[0],
                'sugestao_lideranca_grupo': best_suggested_group,
                'aptidao_score_geral': round(best_aptitude_score, 2),
                'justificativa_bagagem': row['bagagem_contribuicao_cleaned'],
                'justificativa_topico_lda': f"Tópico {int(row['main_topic'])}" if not pd.isna(row['main_topic']) else "N/A",
                'justificativa_sentimento': row['objetivo_proposito_sentiment'] + "/" + row['bagagem_contribuicao_sentiment'],
            })

df_potential_leaders_sim = pd.DataFrame(potential_support_sim_insights)

print("\n--- Sugestões de Potenciais Líderes de Suporte (SIMULADO NO NOTEBOOK) ---")
if not df_potential_leaders_sim.empty:
    print(df_potential_leaders_sim.sort_values(by='aptidao_score_geral', ascending=False).head())
else:
    print("Nenhum potencial líder de suporte identificado com a lógica atual para os grupos carentes.")

print("\n--- FIM DA ANÁLISE INTERATIVA ---")
print("Após refinar o TOPIC_TO_GROUP_APTITUDE_MAP_NOTEBOOK, atualize o dicionário TOPIC_TO_GROUP_APTITUDE_MAP em src/config.py.")
print("Em seguida, execute 'python run_eda.py' e depois 'streamlit run src/app/main.py' para ver as mudanças no dashboard.")